Notebook utilizado para apresentar uma breve introdução ao processamento de sinais.
Criado para a disciplina Geofísica I da Universidade Federal de Uberlândia.
O uso e a reprodução são livres para fins educacionais, pede-se apenas a citação da fonte.

[![License: GPL v3](https://img.shields.io/badge/License-GPLv3-blue.svg)](https://www.gnu.org/licenses/gpl-3.0)

Prof. Dr. Emerson Rodrigo Almeida<br>
Universidade Federal de Uberlândia<br>
Instituto de Geografia - Curso de Geologia<br>
emerson.almeida@ufu.br<br>

<p>&nbsp;</p>
Última modificação: 25/02/2021

# Instruções iniciais

Neste notebook você irá aprender como a digitalização de um sinal analógico é feita corretamente a partir dos intervalos de amostragem. 

Não se preocupe em compreender o código em si, não é obrigatório que você conheça a programação para obter e interpretar os resultados da demonstração. Mesmo assim, sinta-se livre caso queira perguntar qualquer coisa para aprender mais sobre o código e sua elaboração.

<p>&nbsp;</p>

### Passo 01 - Preparação

Primeiramente vamos preparar o notebook com os módulos e funções que precisaremos no decorrer da demonstração. A célula abaixo contém as funções utilizadas nos cálculos e as funções utilizadas para gerar as figuras. Você pode executar o código na célula clicando sobre ela e em seguida clicando no botão **RUN** na barra do Jupyter Notebook ou pressionando as teclas **SHIFT+ENTER** no seu teclado.

<p>&nbsp;</p>

**Importante!** Nem todas as células irão retornar algum tipo de mensagem ou figura após a execução. Isto é normal e não significa que a execução tenha falhado. Perceba o símbolo **In [   ]:** próximo ao canto superior esquerdo de cada célula. Ao iniciar a execução de uma delas você verá um asterisco ($*$) aparecer dentro dos colchetes. Você saberá que a execução ocorreu sem problemas quando o $*$ dentro dos colchetes for substituído por um número e nenhuma mensagem de erro aparecer abaixo da célula executada. 

In [ ]:
import modulos.amostragem2 as am

### Passo 2 - Características do sinal analógico

Aqui iremos configurar as características que queremos no nosso sinal analógico. Nosso sinal analógico será caracterizado por uma sobreposição de cinco funções cosseno, cada uma possuindo suas próprias características de amplitude ($A_1, A_2, ..., A_5$), dada em unidades de amplitude (ua) e de frequência ($f_1, f_2, ..., f_5$), dada em Hertz (Hz). Assim, teremos:

$$ s_1(t) = A_1 cos(2 \pi f_1 t)$$
$$ s_2(t) = A_2 cos(2 \pi f_2 t)$$
$$ s_3(t) = A_3 cos(2 \pi f_3 t)$$
$$ s_4(t) = A_4 cos(2 \pi f_4 t)$$
$$ s_5(t) = A_5 cos(2 \pi f_5 t)$$

$$ y(t) = s_1(t) + s_2(t) + s_3(t) + s_4(t) + s_5(t)$$

Cada função cosseno $s_n(t)$ apresenta uma *componente* de sinal que constitui o sinal $y(t)$, e todas as componentes são definidas dentro de uma mesma janela temporal de 5.0 s. As componentes de sinal que iremos utilizar têm amplitudes $A_1$ = 1.0 ua, $A_2$ = 0.5 ua, $A_3$ = 1.5 ua, $A_4$ = 2.75 ua e $A_5$ = 0.8 ua e frequências $f_1$ = 1.0 Hz, $f_2$ = 3.0 Hz, $f_3$ = 15.0 Hz, $f_4$ = 0.50 Hz e $f_5$ = 7.0 Hz. Execute a célula abaixo para estabelecer as configurações dos parâmetros do sinal analógico e visualizá-lo.

In [ ]:
a1 = 1.00      # amplitude da componente s_1(t) em unidades de amplitude
f1 = 1.00      # frequência da componente s_1(t) em Hertz

a2 = 0.50      # amplitude da componente s_2(t) em unidades de amplitude
f2 = 3.00      # frequência da componente s_2(t) em Hertz

a3 = 1.50      # amplitude da componente s_3(t) em unidades de amplitude
f3 = 15.0      # frequência da componente s_3(t) em Hertz

a4 = 2.75      # amplitude da componente s_4(t) em unidades de amplitude
f4 = 0.50      # frequência da componente s_4(t) em Hertz

a5 = 0.80      # amplitude da componente s_5(t) em unidades de amplitude
f5 = 7.00      # frequência da componente s_5(t) em Hertz

janela_tempo = 5.0              # janela temporal em segundos


funcao_sinal = am.gera_funcao_ref([a1, a2, a3, a4, a5], [f1, f2, f3, f4, f5], janela_tempo)
sinal_analogico, t_analogico = am.gera_sinal_analogico(janela_tempo, funcao_sinal)
am.plota_sinal(t_analogico, sinal_analogico, janela_tempo, "Figura 1. Representação do sinal analógico original")

**Perguntas:** Você consegue identificar visualmente todas as componentes neste sinal? Dica: se você quiser visualizar uma única componente do sinal basta configurar a amplitude de cada uma das outras quatro componentes para zero na primeira célula deste passo. Por exemplo, para visualizar apenas a componente $s_3(t)$, configure as amplitudes $A_1$, $A_2$, $A_4$ e $A_5$ para 0.0. Não se esqueça de restabelecer os valores padrão de amplitude de cada componente antes de continuar a demonstração.

### Passo 03 - Amostragem

A amostragem do sinal com múltiplas componentes obedece aos mesmos critérios estabelecidos pela Teoria da Amostragem que vimos anteriormente. Porém, antes tínhamos apenas uma componente e agora temos várias. Qual delas deverá ser escolhida? A resposta para esta pergunta é: a frequência mais alta que você quer registrar. Tome como exemplo a componente $s_3(t)$. Ela tem frequência de 15 Hz, que é a mais alta de todas e a única acima de 10 Hz. Se for do seu interesse incluir esta frequência na amostragem você deve considerá-la no processo de amostragem. Por outro lado, se você considerar que esta frequência esteja associada a uma componente de ruído e não interessa ao registro do seu sinal, deverá considerar como frequência máxima a frequência do sinal $s_5(t)$, que é de 7.0 Hz. 

A frequência máxima capaz de ser amostrada por um determinado intervalo de amostragem é denominada **frequência de Nyquist** ($f_N$). O intervalo de amostragem necessário para registrar esta frequência pode ser obtido da mesma forma que fizemos anteriormente, através do critério estabelecido pelo Teorema da Amostragem:

$$\Delta t_{am_{N}} = \frac{1}{f_{am_{N}}} = \frac{1}{2 * f_N}$$

Vamos então definir esta frequência de amostragem para nosso sinal executando a célula abaixo:

In [ ]:
f_N = 15.0          # frequência de Nyquist, em Hz
f_am = 2 * f_N      # frequência de amostragem

sinal_amostrado, t_amostrado = am.amostra_sinal(janela_tempo, 1/f_am, funcao_sinal)
am.plota_amostragem(t_analogico, sinal_analogico, t_amostrado, sinal_amostrado, janela_tempo, \
                 "Figura 2. Sinal amostrado a uma frequência de " + str(f_am) + " amostras por segundo")

**Pergunta:** Qual o intervalo de amostragem utilizado?

### Passo 04 - Recuperação do sinal

Quando o sinal é amostrado corretamente é possível recuperá-lo através de sistemas dedicados, que fogem ao escopo desta demonstração. Aqui exemplificaremos a recuperação do sinal através de uma simples interpolação.

Quando registramos as amostras de um sinal, as informações sobre a amplitude do sinal e sobre os intantes de tempo em que estas amplitudes foram medidas são tudo o que temos para trabalhar. Não há informação alguma sobre o comportamento do sinal, que pode ser uma função cosseno, uma função seno, uma sobreposição de ambas, ou qualquer outra coisa. Assim como um software de análise de dados geofísicos não tem ideia sobre o comportamento do sinal verdadeiro, não há qulquer informação pré-determinada sobre as características do sinal original na interpolação realizada nesta demonstração. 

Execute a célula abaixo para visualizar uma compraração entre o sinal recuperado (em azul) e o sinal original (em vermelho).

In [ ]:
sinal_rec, t_rec = am.recupera_sinal(t_amostrado, sinal_amostrado)
am.plota_representacao(t_amostrado, sinal_amostrado, t_rec, sinal_rec, janela_tempo, "Figura 3. Sinal recuperado da amostragem")
am.plota_comparacao(t_analogico, sinal_analogico, t_rec, sinal_rec, janela_tempo, \
                 "Figura 4. Comparação entre o sinal original e o sinal recuperado da amostragem")

**Pergunta:** Você considera que a recuperação foi bem feita? Qual a frequência máxima do sinal original? Qual a frequência máxima que você consegue identificar no sinal recuperado?